In [1]:
import torch
import pickle
import numpy as np
import pandas as pd
import os

from os.path import dirname



root_path = dirname(os.getcwd()) + "/HGNN_NA"

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "BPI_Challenge_2012_A"

In [3]:
tab_all = pd.read_csv(data_dir_processed+dataset+"_processed_all.csv")
print(tab_all.head())
list_activities = list(tab_all["Activity"].unique())
list_org_resource = list(tab_all['org:resource'].unique())
list_org_resource_roles = list(tab_all["org:resource:role"].unique())

   org:resource lifecycle:transition           Activity  time:timestamp  \
0           112             COMPLETE        A_SUBMITTED        0.000000   
1           112             COMPLETE  A_PARTLYSUBMITTED        0.000000   
2           112             COMPLETE      A_PREACCEPTED        3.970292   
3         10862             COMPLETE         A_ACCEPTED       10.592602   
4         10862             COMPLETE        A_FINALIZED       10.596260   

   case:REG_DATE  CaseID  case:AMOUNT_REQ org:resource:role  
0       8.881836  173688         9.903488            Role 3  
1       8.881836  173688         9.903488            Role 3  
2       8.881836  173688         9.903488            Role 3  
3       8.881836  173688         9.903488            Role 1  
4       8.881836  173688         9.903488            Role 1  


In [ ]:
tab_all['org:resource:role'].value_counts()

In [4]:
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [5]:
with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_newF_onehot_lessEdgesWclass_wRole.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_newF_onehot_lessEdgesWclass_wRole.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_newF_onehot_lessEdgesWclass_wRole.pkl", "rb") as f:
    X_test = pickle.load(f)

In [ ]:

with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_SEMIFINAL_NOATTR.pkl", "rb") as f:
    X_train = pickle.load(f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_SEMIFINAL_NOATTR.pkl", "rb") as f:
    X_valid = pickle.load(f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_SEMIFINAL_NOATTR.pkl", "rb") as f:
    X_test = pickle.load(f)

In [6]:

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToUndirected, NormalizeFeatures

transform = ToUndirected()

with torch.no_grad():
        for i in range(len(X_train)):
                X_train[i] = transform(X_train[i])
        for i in range(len(X_valid)):
                X_valid[i] = transform(X_valid[i])
        for i in range(len(X_test)):
                X_test[i] = transform(X_test[i])
    






In [7]:
edge_types = set()
node_types = set()
for i in range(len(X_train)):
    n, edge_type = X_train[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_valid)):
    n, edge_type = X_valid[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
for i in range(len(X_test)):
    n, edge_type = X_test[i].metadata()
    for x in n:
        node_types.add(x)
    for x in edge_type:
        edge_types.add(x)
node_types = list(node_types)
edge_types = list(edge_types)

In [ ]:
#node_types, edge_types = X_train[0].metadata()

In [8]:
def get_weights(load, key):
    weights = []
    
    cl_train = [0 for _ in tab_all[key].unique()]
    
    print(cl_train)
    
    for i,x in enumerate(load):

        
        classes = x.y[key]

        # print(classes)
        
        for c in list(classes):
            try:
                cl_train[c] +=1
            except KeyError:
                cl_train[c] = 1
    s = sum(cl_train)
    
    print(cl_train)
    
    weights = [s/x if x > 0 else 0 for x in cl_train]

    # weights = [0.7,0.7,1,0.7,0.7,0.7,0.7,0.7,0.7,0.7]
    weights = torch.tensor(weights, device=device)
    print(weights)
    return weights
    

In [25]:
edge_features_dims = {}
for i in range(len(X_train)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_valid)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
for i in range(len(X_test)):
    for k in X_train[i].edge_attr_dict.keys():
        edge_features_dims[k] = X_train[i].edge_attr_dict[k].shape[1]
edge_features_dims

{('org:resource', 'related_to', 'org:resource'): 2,
 ('Activity', 'followed_by', 'Activity'): 2,
 ('time:timestamp', 'related_to', 'time:timestamp'): 2,
 ('org:resource:role', 'related_to', 'org:resource:role'): 2}

In [10]:
act_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "Activity")
res_roles_weights = get_weights(DataLoader(X_train, batch_size=1024, shuffle=False), "org:resource:role")


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2683, 1238, 1238, 1586, 4155, 2636, 0, 3851, 1238, 0]
tensor([ 6.9419, 15.0444, 15.0444, 11.7434,  4.4826,  7.0656,  0.0000,  4.8364,
        15.0444,  0.0000], device='cuda:0')
[0, 0, 0]
[9394, 4215, 5016]
tensor([1.9826, 4.4187, 3.7131], device='cuda:0')


## Hyperopt

In [11]:
from ax.service.managed_loop import optimize

In [12]:
from torch_geometric.nn import (
    HeteroConv,
    global_mean_pool,
    GATv2Conv,
)
from torch.nn import (
    ModuleList,
    Module,
    Sequential,
    Linear,
  )
from typing_extensions import Self

In [13]:
class HGNN(Module):

    def __init__(self, out_act, out_org_resource,out_org_resource_role,nodes_relations, relations_with_features, parameters) -> Self:  # type: ignore
        super().__init__()

        # List of convolutional layers
        
        hid = parameters["hid"]
        layers = parameters["layers"]
        aggregation = parameters["aggregation"]
        n_heads = parameters["heads"]
     
        
        self.convs = ModuleList()
        for _ in range(layers):
            conv = HeteroConv(
                {
                    relation: (
                        GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid, heads=n_heads, concat=False)
                        if relation not in relations_with_features
                        else GATv2Conv((-1,-1), add_self_loops=False, out_channels=hid,heads=n_heads, edge_dim=relations_with_features[relation], concat=False)
                    )
                    for relation in nodes_relations
                },
                aggr=aggregation,
            )

            self.convs.append(conv)

        self.fc_NA = Linear(hid, out_act)
            #Linear(hid*n_heads, out_act)
       
        self.fc_timestamp = Linear(hid, 1)
            #Linear(hid*n_heads, 1)
       
        self.fc_org_resource = Linear(hid, out_org_resource)
            #Linear(hid*n_heads, out_org_resource)
        
        self.fc_org_resource_role = Linear(hid, out_org_resource_role)
        
        

    def forward(self, batch):

        for i in range(len(self.convs)):
            batch.x_dict = self.convs[i]( 
                batch.x_dict, batch.edge_index_dict, batch.edge_attr_dict
            )

            batch.x_dict = {key: x.relu() for key, x in batch.x_dict.items()}

        

        act = global_mean_pool(batch.x_dict["Activity"], batch["Activity"].batch)
        tm = global_mean_pool(batch.x_dict["time:timestamp"], batch["time:timestamp"].batch)
        ores = global_mean_pool(batch.x_dict["org:resource"], batch["org:resource"].batch)
        ores_roles = global_mean_pool(batch.x_dict["org:resource:role"], batch["org:resource:role"].batch)

        activities = self.fc_NA(act)
        timestamps = self.fc_timestamp(tm).reshape(1,-1)[0]
        org_resources = self.fc_org_resource(ores)
        org_resources_role = self.fc_org_resource_role(ores_roles)

        return [activities, timestamps, org_resources, org_resources_role]

In [14]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_f1_score

In [15]:
import torch.nn as nn

In [16]:
import time

In [17]:
from copy import deepcopy


def train_hgnn(config):
    print(config)
    
   
    
    net = HGNN(
        parameters=config,
        out_act=len(list_activities),
        out_org_resource=len(list_org_resource),
        out_org_resource_role=len(list_org_resource_roles),
        nodes_relations=edge_types,
        relations_with_features=edge_features_dims,
    )
    net.to(device)

    act_loss = nn.CrossEntropyLoss(act_weights)
    timestamp_loss = nn.L1Loss()
    org_resource_loss = nn.CrossEntropyLoss()
    org_resource_roles_loss = nn.CrossEntropyLoss()

    
    train_loader = DataLoader(X_train, batch_size=config["batch_size"], shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=config["batch_size"], shuffle=True)
    
    
    #if config["optim"] == "Adam":
    optimizer = torch.optim.Adam(net.parameters(), lr=config["lr"])
    #else:
    #    with torch.no_grad():
    #        for _,x in enumerate(train_loader):
    #            x.to(device)
    #            _, _, _ = net(x)
    #            break
    #    optimizer = torch.optim.Adagrad(net.parameters(), lr=config["lr"])
        
    
    

    best_model = None
    best_loss = 0
    patience = 5
    pat_count = 0
    
    torch.cuda.empty_cache()
    
    for epoch in range(0, 20):
        start_time = time.time() 

        print(f"Epoch: {epoch}\n")
        
        
        net.train()
        for _, x in enumerate(train_loader):
            x = x.to(device)

            activities_labels = x.y["Activity"]
            timestamp_labels = x.y["time:timestamp"]
            org_resource_labels = x.y["org:resource"]
            org_resource_roles_labels = x.y["org:resource:role"]
            
            optimizer.zero_grad()
            outputs_act, outputs_timestamp, outputs_org_resource, outputs_org_resource_roles = net(x)
            
            act_loss_step = act_loss(outputs_act, activities_labels)
            org_resource_loss_step = org_resource_loss(outputs_org_resource, org_resource_labels)
            org_resource_roles_loss_step = org_resource_roles_loss(outputs_org_resource_roles, org_resource_roles_labels)
            timestamp_loss_step = timestamp_loss(outputs_timestamp, timestamp_labels)
            
            total_loss = (act_loss_step + timestamp_loss_step + org_resource_loss_step + org_resource_roles_loss_step)
            total_loss.backward()
            optimizer.step()
            

        
        
        prediction_activity = []
        target_activity = []
        
        prediction_resource = []
        target_resource = []
        
        prediction_resource_roles = []
        target_resource_roles = []
        
        avg_mse_timestamp = []
        
        
        
        running_total_loss = []
        
        net.eval()
        with torch.no_grad():
            for i, x in enumerate(valid_loader):
                x = x.to(device)

                activities_labels = x.y["Activity"]
                timestamp_labels = x.y["time:timestamp"]
                org_resource_labels = x.y["org:resource"]
                org_resource_roles_labels = x.y["org:resource:role"]
                
                outputs_act, outputs_timestamp, outputs_org_resource, outputs_org_resource_roles = net(x)
                
                act_loss_step = act_loss(outputs_act, activities_labels)
                org_resource_loss_step = org_resource_loss(outputs_org_resource, org_resource_labels)
                org_resource_roles_loss_step = org_resource_roles_loss(outputs_org_resource_roles, org_resource_roles_labels)
                timestamp_loss_step = timestamp_loss(outputs_timestamp, timestamp_labels)
                
                running_total_loss.append(act_loss_step.item() + org_resource_loss_step.item() + timestamp_loss_step.item() + org_resource_roles_loss_step.item())
                

                
                preds_act = torch.argmax(torch.softmax(outputs_act, dim=1), 1)
                prediction_activity.append(preds_act)
                target_activity.append(activities_labels)
                
                
                preds_org_resource = torch.argmax(torch.softmax(outputs_org_resource, dim=1), 1)
                prediction_resource.append(preds_org_resource)
                target_resource.append(org_resource_labels)
                
                preds_org_resource_roles = torch.argmax(torch.softmax(outputs_org_resource_roles, dim=1), 1)
                prediction_resource_roles.append(preds_org_resource_roles)
                target_resource_roles.append(org_resource_roles_labels)
                
                
                
                avg_mse_timestamp.append(timestamp_loss_step.item())
        
        
        
                
                
        prediction_activity = torch.cat(prediction_activity)
        target_activity = torch.cat(target_activity)
        
        prediction_resource = torch.cat(prediction_resource)
        target_resource = torch.cat(target_resource)
        
        prediction_resource_roles = torch.cat(prediction_resource_roles)
        target_resource_roles = torch.cat(target_resource_roles)
        
        
        macro_f1_activity = multiclass_f1_score(prediction_activity, target_activity, num_classes=len(list_activities), average="macro")
        
        act_accuracy = multiclass_accuracy(prediction_activity, target_activity, num_classes=len(list_activities))
        resource_accuracy = multiclass_accuracy(prediction_resource, target_resource, num_classes=len(list_org_resource))
        resource_roles_accuracy = multiclass_accuracy(prediction_resource_roles, target_resource_roles, num_classes=len(list_org_resource_roles))
        
        MAE_timestamp = sum(avg_mse_timestamp) / len(avg_mse_timestamp)
        
        val_loss = sum(running_total_loss) / len(running_total_loss)
        
        
        
        print(f"\nVALIDATION")
        print("ACTIVITY: acc {:.4f} MacroF1 {:.4f}".format(act_accuracy, macro_f1_activity))
        print("TIMESTAMP: MAE {:.4f}".format(MAE_timestamp))
        print("RESOURCE: acc {:.4f}".format(resource_accuracy))
        print("RESOURCE_ROLE: acc {:.4f}".format(resource_roles_accuracy))
        print("TOTAL_LOSS: {:.4f}".format(val_loss))
        print("epoch time {}s\n".format(time.time() - start_time))
        
        
        
        if epoch == 0:
            best_model= deepcopy(net)
            best_loss = val_loss
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = deepcopy(net)
                pat_count = 0
                print("new best model found")
            if pat_count == patience:
                print("Validation performance didn\'t improve for {} epochs. Training stops.".format(pat_count))
                return best_model
        pat_count += 1    


    return best_model
            


    

In [18]:
def test_hgnn(net):
    test_loader = DataLoader(X_train, batch_size=128, shuffle=False)
    
    act_loss = nn.CrossEntropyLoss(act_weights)
    timestamp_loss = nn.L1Loss()
    resource_loss = nn.CrossEntropyLoss()
    resource_roles_loss = nn.CrossEntropyLoss() 
    
    prediction_activity = []
    target_activity = []
    
    prediction_resource = []
    target_resource = []
    
    prediction_resource_roles = []
    target_resource_roles = []
    
    avg_mse_timestamp = []
    
    total_loss = []
        
    net.eval()
    with torch.no_grad():
        for i, x in enumerate(test_loader):
            x = x.to(device)
            activities_labels = x.y["Activity"]
            timestamp_labels = x.y["time:timestamp"]
            org_resource_labels = x.y["org:resource"]
            org_resource_roles_labels = x.y["org:resource:role"]
            
            outputs_act, outputs_timestamp, outputs_org_resource, outputs_org_resource_roles = net(x)
            
     
            timestamp_loss_step = timestamp_loss(outputs_timestamp, timestamp_labels)
            act_loss_step = act_loss(outputs_act, activities_labels)
            resource_loss_step = resource_loss(outputs_org_resource, org_resource_labels)
            org_resource_roles_loss_step = resource_roles_loss(outputs_org_resource_roles, org_resource_roles_labels)
            
            preds_act = torch.argmax(torch.softmax(outputs_act, dim=1), 1)
            prediction_activity.append(preds_act)
            target_activity.append(activities_labels)
            
            
            preds_org_resource = torch.argmax(torch.softmax(outputs_org_resource, dim=1), 1)
            prediction_resource.append(preds_org_resource)
            target_resource.append(org_resource_labels)
            
            preds_org_resource_roles = torch.argmax(torch.softmax(outputs_org_resource_roles, dim=1), 1)
            prediction_resource_roles.append(preds_org_resource_roles)
            target_resource_roles.append(org_resource_roles_labels)
            
            
            
            avg_mse_timestamp.append(timestamp_loss_step.item())
            
            total_loss.append(timestamp_loss_step.item() + act_loss_step.item() + resource_loss_step.item() + org_resource_roles_loss_step.item())
            
            
    
    prediction_activity = torch.cat(prediction_activity)
    target_activity = torch.cat(target_activity)
    
    prediction_resource = torch.cat(prediction_resource)
    target_resource = torch.cat(target_resource)
    
    prediction_resource_roles = torch.cat(prediction_resource_roles)
    target_resource_roles = torch.cat(target_resource_roles)
        
    act_accuracy = multiclass_accuracy(prediction_activity, target_activity, num_classes=len(list_activities))
    resource_accuracy = multiclass_accuracy(prediction_resource, target_resource, num_classes=len(list_org_resource))
    resource_roles_accuracy = multiclass_accuracy(prediction_resource_roles, target_resource_roles, num_classes=len(list_org_resource_roles))
    MAE_timestamp = sum(avg_mse_timestamp) / len(avg_mse_timestamp)
    
    Average_total_loss = sum(total_loss) / len(total_loss)
    
    macro_f1_activity = multiclass_f1_score(prediction_activity, target_activity, num_classes=len(list_activities), average="macro")
    
    return { "MacroF1Activity" : macro_f1_activity.item(), "activity_accuracy"  : act_accuracy.item(), "resource_accuracy" : resource_accuracy.item(), "resource_roles_accuracy" : resource_roles_accuracy.item(), "MAE_timestamp" : MAE_timestamp, "AVG_total_loss" : Average_total_loss} 

In [19]:
def train_evaluate(config):
    trained_net = train_hgnn(config)
    return test_hgnn(trained_net)

In [20]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "hid", "type": "choice", "values": [128], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "layers", "type": "choice", "values": [2, 3, 4, 5], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "layers", "type": "choice", "values": [2], "value_type": "int", "is_ordered" : True, "sort_values":False},
        {"name": "lr", "type": "range", "bounds": [1e-4, 1e-1], "value_type": "float", "log_scale": True},
        {"name": "batch_size", "type": "choice", "values": [256, 1024], "value_type": "int", "is_ordered" : True,"sort_values":False}, 
        {"name": "heads", "type": "choice", "values": [1,4], "value_type": "int", "is_ordered" : True,"sort_values":False},
        #{"name": "heads", "type": "choice", "values": [1], "value_type": "int", "is_ordered" : True,"sort_values":False},
        
        #{"name": "aggregation", "type" : "choice", "values" :["sum", "mean", "max"], "value_type" : "str"}
        {"name": "aggregation", "type" : "choice", "values" :["max"], "value_type" : "str"},
     
    ],
  
    evaluation_function=train_evaluate,
    objective_name='AVG_total_loss',
    arms_per_trial=1,
    minimize = True,
    random_seed = 123,
    total_trials = 20
)

print(best_parameters)
means, covariances = values
print(means)
print(experiment)

[INFO 01-07 17:28:19] ax.service.utils.instantiation: Choice parameter hid contains only one value, converting to a fixed parameter instead.
[INFO 01-07 17:28:19] ax.service.utils.instantiation: Choice parameter layers contains only one value, converting to a fixed parameter instead.
[INFO 01-07 17:28:19] ax.service.utils.instantiation: Choice parameter aggregation contains only one value, converting to a fixed parameter instead.
[INFO 01-07 17:28:19] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[FixedParameter(name='hid', parameter_type=INT, value=128), FixedParameter(name='layers', parameter_type=INT, value=2), RangeParameter(name='lr', parameter_type=FLOAT, range=[0.0001, 0.1], log_scale=True), ChoiceParameter(name='batch_size', parameter_type=INT, values=[256, 1024], is_ordered=True, sort_values=False), ChoiceParameter(name='heads', parameter_type=INT, values=[1, 4], is_ordered=True, sort_values=False), FixedParameter(name='aggregation', parameter_ty

{'lr': 0.0001865322718407359, 'batch_size': 1024, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
Epoch: 0




VALIDATION
ACTIVITY: acc 0.2629 MacroF1 0.1021
TIMESTAMP: MAE 11.9874
RESOURCE: acc 0.2648
RESOURCE_ROLE: acc 0.5107
TOTAL_LOSS: 18.9187
epoch time 12.660966157913208s

Epoch: 1




VALIDATION
ACTIVITY: acc 0.1454 MacroF1 0.0317
TIMESTAMP: MAE 8.4879
RESOURCE: acc 0.2648
RESOURCE_ROLE: acc 0.5107
TOTAL_LOSS: 15.0804
epoch time 12.187115669250488s

new best model found
Epoch: 2




VALIDATION
ACTIVITY: acc 0.2921 MacroF1 0.2131
TIMESTAMP: MAE 4.6909
RESOURCE: acc 0.2648
RESOURCE_ROLE: acc 0.5107
TOTAL_LOSS: 11.1291
epoch time 12.34177851676941s

new best model found
Epoch: 3



...


VALIDATION
ACTIVITY: acc 0.6646 MacroF1 0.6229
TIMESTAMP: MAE 0.1096
RESOURCE: acc 0.5354
RESOURCE_ROLE: acc 0.7860
TOTAL_LOSS: 3.3879
epoch time 14.522538423538208s

Epoch: 13




VALIDATION
ACTIVITY: acc 0.6707 MacroF1 0.6354
TIMESTAMP: MAE 0.1500
RESOURCE: acc 0.5380
RESOURCE_ROLE: acc 0.7860
TOTAL_LOSS: 3.3905
epoch time 14.358592510223389s

Epoch: 14




VALIDATION
ACTIVITY: acc 0.6671 MacroF1 0.6235
TIMESTAMP: MAE 0.3775
RESOURCE: acc 0.5380
RESOURCE_ROLE: acc 0.7860
TOTAL_LOSS: 3.6397
epoch time 14.491517066955566s

Validation performance didn't improve for 5 epochs. Training stops.


[INFO 01-07 18:53:55] ax.core.experiment: Attached data has some metrics ({'activity_accuracy', 'MAE_timestamp', 'resource_roles_accuracy', 'MacroF1Activity', 'resource_accuracy'}) that are not among the metrics on this experiment. Note that attaching data will not automatically add those metrics to the experiment. For these metrics to be automatically fetched by `experiment.fetch_data`, add them via `experiment.add_tracking_metric` or update the experiment's optimization config.
[ERROR 01-07 18:53:55] ax.core.observation: Data contains metric MAE_timestamp that has not been added to the experiment. You can either update the `optimization_config` or attach it as a tracking metric using `Experiment.add_tracking_metrics` or `AxClient.add_tracking_metrics`. Ignoring all data for metric MAE_timestamp.
NoneType: None
[ERROR 01-07 18:53:55] ax.core.observation: Data contains metric MacroF1Activity that has not been added to the experiment. You can either update the `optimization_config` or a

{'lr': 0.004111734182057972, 'batch_size': 256, 'heads': 1, 'hid': 128, 'layers': 2, 'aggregation': 'max'}
{'AVG_total_loss': 3.066705279579242}
Experiment(None)


In [21]:
from ax.service.utils.report_utils import exp_to_df

results = exp_to_df(experiment)

[WARNING 01-07 18:56:09] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [24]:
results.sort_values(by="activity_accuracy")

,trial_index,arm_name,trial_status,generation_method,AVG_total_loss,MAE_timestamp,MacroF1Activity,activity_accuracy,resource_accuracy,resource_roles_accuracy,lr,batch_size,heads,hid,layers,aggregation
0,0,0_0,COMPLETED,Sobol,4.662463,0.124686,0.465442,0.611383,0.319463,0.754792,0.000187,1024,1,128,2,max
16,16,16_0,COMPLETED,BoTorch,3.634319,0.468481,0.599465,0.614550,0.512215,0.789906,0.100000,256,4,128,2,max
1,1,1_0,COMPLETED,Sobol,3.185271,0.215865,0.610737,0.642201,0.565262,0.791678,0.020423,256,4,128,2,max
13,13,13_0,COMPLETED,BoTorch,4.043543,0.124550,0.591317,0.643329,0.321772,0.790658,0.000100,256,4,128,2,max
7,7,7_0,COMPLETED,BoTorch,3.580545,0.302930,0.594533,0.646013,0.481020,0.791785,0.100000,1024,1,128,2,max
17,17,17_0,COMPLETED,BoTorch,3.130027,0.141115,0.597251,0.648752,0.566121,0.791839,0.003454,256,4,128,2,max
3,3,3_0,COMPLETED,Sobol,3.116581,0.129809,0.601565,0.650792,0.566819,0.791839,0.002055,256,4,128,2,max
6,6,6_0,COMPLETED,BoTorch,3.105021,0.109936,0.608279,0.651329,0.563973,0.791839,0.005120,256,4,128,2,max
4,4,4_0,COMPLETED,Sobol,3.570439,0.111962,0.614537,0.652295,0.438819,0.790658,0.001014,1024,4,128,2,max
15,15,15_0,COMPLETED,BoTorch,3.179014,0.119093,0.611515,0.653960,0.556832,0.791839,0.003216,1024,4,128,2,max


In [ ]:
results = results.sort_values(by="AVG_total_loss")

In [ ]:
results.to_csv("SAGECONV_resROLES.csv", sep=",")